# pattern for multiple annex styles in one and entity types

In [35]:
#table annex with numbers
import pandas
import os
import requests
import re
from bs4 import BeautifulSoup

url = 'https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:32011D0236'
response = requests.get(url)
print(response.status_code)
data = response.text
data

200


'<?xml version="1.0" encoding="UTF-8"?>\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML//EN" "xhtml-strict.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<!-- CONVEX # converter_version:9.15.0 # generated_on:20240304-2034 # ELI version:0.10 -->\r\n<head><meta name="format-detection" content="telephone=no"/><meta http-equiv=\'X-UA-Compatible\' content=\'IE=edge\'/><meta name=\'viewport\' content=\'width=device-width, initial-scale=1\'><meta name=\'WT.z_usr_lan\' content=\'EN\'>\n   <meta http-equiv="content-type" content="text/html; charset=utf-8"/>\n   <script type="text/javascript" src="/eurlex-frontoffice/ruxitagentjs_ICANVfgqrux_10289240325103055.js" data-dtconfig="app=47d4c64c3b67ec69|agentId=bdb94867a953efdd|featureHash=ICANVfgqrux|rdnt=1|uxrgce=1|bp=3|cuc=m097nmfl|mel=100000|mb=null|dpvc=1|iub=null|ssv=4|lastModification=1721139136808|tp=500,50,0|agentUri=/eurlex-frontoffice/ruxitagentjs_ICANVfgqrux_10289240325103055.js|reportUrl=/eurlex-frontoffice/rb_39a3e95b-5423-482c-879

*get the EU sanction list*

In [ ]:
#check for list updates, removals or deletions
action_intended = re.findall(r'<p class="(oj-normal|oj-bold)">\s*(.*?\s*(?:removed|updated|deleted|replaced)\s*.*?)\s*</p>',data)
action_intended

In [36]:
#get the document information
url= re.findall(r'href="(.*?\.eu/legal-content/EN/TXT/HTML/.*?)"', data)
publication_date = re.findall(r'<p class="oj-hd-date">(\d{1,2}\.\d{1,2}\.\d{4})', data)
title = re.findall(r'class="oj-doc-ti"[^>]*>([\s\S]*?)<\/p>', data)
celex= re.findall(r'uri=CELEX%([^"]+)', data)
decision_match = re.findall(r'(COUNCIL IMPLEMENTING.*?)</p', data)
legal_decision = re.findall(r'<p[^>]*>(.*?amending|implementing.*?)</p>',data)
legal_base = re.findall(r'<p[^>]*>(.*?regard to.*?)</p>',data)
print(url),print(publication_date),print(title),print(celex),print(decision_match),print(legal_decision),print(legal_base)

['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32011D0236']
['14.4.2011']
['\n            COUNCIL IMPLEMENTING DECISION 2011/236/CFSP', 'of 12 April 2011\n         ', 'implementing Decision 2011/137/CFSP concerning restrictive measures in view of the situation in Libya', 'ANNEX I', 'ANNEX II', 'ANNEX III', 'ANNEX IV']
['3A32011D0236']
['COUNCIL IMPLEMENTING DECISION 2011/236/CFSP']
['implementing Decision 2011/137/CFSP concerning restrictive measures in view of the situation in Libya']
['Having regard to Council Decision 2011/137/CFSP of 28 February 2011 concerning restrictive measures in view of the situation in Libya\xa0<a id="ntc1-L_2011100EN.01005801-E0001" href="#ntr1-L_2011100EN.01005801-E0001">(<span class="oj-super oj-note-tag">1</span>)</a>, and in particular Article 8(1) and (2) thereof, in conjunction with Article 31(2) of the Treaty on European Union,']


(None, None, None, None, None, None, None)

In [64]:
#get information on the entity type
##!!!!!!!!!!! needs fixing as it takes the previous span and classifies the entity type wrongly

def extract_entity_type(span):
    # Check for entity type in the preceding tags
    entity_type = None
    if span.find_previous(string=re.compile(r'Entities')):
        entity_type = 'Entity'
    elif span.find_previous(string=re.compile(r'persons')):
        entity_type = 'Person'
    elif span.find_previous(string=re.compile(r'Persons')):
        entity_type = 'Person'
    return entity_type

soup = BeautifulSoup(data, 'html.parser')

# Find all rows in the table
rows = soup.find_all('tr', class_='oj-table')

# List to store the parsed data
sanction_list = []

# Iterate through the rows and extract the necessary information
for row in rows[1:]:  # Skip the header row
    cells = row.find_all('td')
    if len(cells) < 5:
        continue

    name = cells[1].get_text(strip=True)
    identifying_info = cells[2].get_text(strip=False)  # Extracting the identifying information
    reasons = ' '.join(p.get_text(strip=True) for p in cells[3].find_all('p'))
    date_of_listing = cells[4].get_text(strip=True)

    # Find the entity type
    entity_type_span = cells[1].find_previous('span', class_='oj-bold')
    entity_type = extract_entity_type(entity_type_span)

    sanction_list.append({
        'URL': url,
        'CELEX': celex,
        'law_number': decision_match[0],
        'Publication Date': publication_date,
        'legal_base': legal_decision[0],
        'Name': name,
        'Identifying Information': identifying_info,
        'Reason': reasons,
        'Date of Listing': date_of_listing,
        'Entity Type': entity_type,
        'UN_sanction': None
    })

# Display the results
for person in sanction_list:
    print(person)



{'URL': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32011D0236'], 'CELEX': ['3A32011D0236'], 'law_number': 'COUNCIL IMPLEMENTING DECISION 2011/236/CFSP', 'Publication Date': ['14.4.2011'], 'legal_base': 'implementing Decision 2011/137/CFSP concerning restrictive measures in view of the situation in Libya', 'Name': 'ABDULHAFIZ, Colonel Mas’ud', 'Identifying Information': '\nPosition: Armed Forces Commander\n', 'Reason': '3rd in command of Armed Forces. Significant role in Military Intelligence.', 'Date of Listing': '28.2.2011', 'Entity Type': 'Person', 'UN_sanction': None}
{'URL': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32011D0236'], 'CELEX': ['3A32011D0236'], 'law_number': 'COUNCIL IMPLEMENTING DECISION 2011/236/CFSP', 'Publication Date': ['14.4.2011'], 'legal_base': 'implementing Decision 2011/137/CFSP concerning restrictive measures in view of the situation in Libya', 'Name': 'ABDUSSALAM, Abdussalam Mohammed', 'Identifying Information

In [63]:
# Parse the HTML content
soup = BeautifulSoup(data, 'html.parser')

# Find all rows in the table
rows = soup.find_all('tr', class_='oj-table')

# List to store the parsed data
sanction_list = []

# Iterate through the rows and extract the necessary information
for row in rows[1:]:  # Skip the header row
    cells = row.find_all('td')
    if len(cells) < 5:
        continue

    name = cells[1].get_text(strip=True)
    identifying_info = cells[2].get_text(strip=False) # Extracting the identifying information
    reasons = ' '.join(p.get_text(strip=True) for p in cells[3].find_all('p'))
    date_of_listing = cells[4].get_text(strip=True)
    
    sanction_list.append({
        'URL': url,
        'CELEX':celex,
        'law_number' :decision_match[0],
        'Publication Date': publication_date,
        'legal_base' : legal_decision[0],
        'Name': name,
        'Identifying Information': identifying_info,
        'Reason': reasons,
        'Date of Listing': date_of_listing,
        'UN_sanction': None
    })

# Display the results
for person in sanction_list:
    print(person)

{'URL': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32011D0236'], 'CELEX': ['3A32011D0236'], 'law_number': 'COUNCIL IMPLEMENTING DECISION 2011/236/CFSP', 'Publication Date': ['14.4.2011'], 'legal_base': 'implementing Decision 2011/137/CFSP concerning restrictive measures in view of the situation in Libya', 'Name': 'ABDULHAFIZ, Colonel Mas’ud', 'Identifying Information': '\nPosition: Armed Forces Commander\n', 'Reason': '3rd in command of Armed Forces. Significant role in Military Intelligence.', 'Date of Listing': '28.2.2011', 'UN_sanction': None}
{'URL': ['https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX%3A32011D0236'], 'CELEX': ['3A32011D0236'], 'law_number': 'COUNCIL IMPLEMENTING DECISION 2011/236/CFSP', 'Publication Date': ['14.4.2011'], 'legal_base': 'implementing Decision 2011/137/CFSP concerning restrictive measures in view of the situation in Libya', 'Name': 'ABDUSSALAM, Abdussalam Mohammed', 'Identifying Information': '\nPosition: Head Coun

In [66]:
#STILL WORK IN PROGRESS
# Define the values to remove from the dictionary
values_to_remove = ['Name', '\nIdentifying information\n', 'Reasons', 'Date of listing']

# Iterate through the dictionary and remove specified values
filtered_data = {key: value for key, value in data.items() if value not in values_to_remove}

# Print the filtered dictionary
print(filtered_data)

AttributeError: 'str' object has no attribute 'items'

*get the UN sanctions*

In [21]:
listing_position = re.findall(r'class="oj-normal">(\d+.*?)',data)
print(listing_position)
name_2 = re.findall(r'<p class="oj-normal">\s*<span class="oj-bold">(.*?)</span>', data)
print(name_2)
date_pattern = re.findall(r'class="oj-normal">Date of UN designation(.*?)</p>',data,re.DOTALL)
print(date_pattern)
identifying_information =re.findall(r'>(.*?Date of Birth:.*?)</p>',data)
print(identifying_information)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '1', '2', '3', '4', '5']
['AL-BAGHDADI, Dr Abdulqader Mohammed', 'DIBRI, Abdulqader Yusef', 'DORDA, Abu Zayd Umar', 'JABIR, Major General Abu Bakr Yunis', 'MATUQ, Matuq Mohammed', 'QADHAF AL-DAM, Sayyid Mohammed', 'QADHAFI, Aisha Muammar', 'QADHAFI, Hannibal Muammar', 'QADHAFI, Khamis Muammar', 'QADHAFI, Mohammed Muammar', 'QADHAFI, Muammar Mohammed Abu Minyar', 'QADHAFI, Mutassim', 'QADHAFI, Saadi', 'QADHAFI, Saif al-Arab', 'QADHAFI, Saif al-Islam', 'AL-SENUSSI, Colonel Abdullah', 'AL QADHAFI, Quren Salih Quren', 'AL KUNI, Colonel Amid Husain', 'QADHAFI, Aisha Muammar', 'QADHAFI, Hannibal Muammar', 'QADHAFI, Khamis Muammar', 'QADHAFI, Muammar Mohammed Abu Minyar', 'QADHAFI, Mutassim', 'QADHAFI, Saif al-Islam', 'DORDA, Abu Zayd Umar', 'JABIR, Major General Abu Bakr Yunis', 'MATUQ, Matuq Mohammed', 'QADHAFI, Mohammed Mua

In [27]:

# Split the data into td blocks
td_blocks = re.split(r'</td>\s*<td valign="top">', data)

# Extract information from each td block
results = []

for block in td_blocks:
    # Extract bold content
    bold_content = re.findall(r'<p class="oj-normal">\s*<span class="oj-bold">(.*?)</span>', block)
    
    # Extract the rest of the information in oj-normal paragraphs
    identifying_information = re.findall(r'<p class="oj-normal">(.*?)</p>', block, re.DOTALL)
    
    if bold_content:
        results.append({
            'CELEX':celex,
            'law_number' :decision_match[0],
            'publication_date': publication_date,
            'legal_base' : legal_decision[0],
            "name": bold_content[0],
            "identifying_information": identifying_information[1],
            "reason":identifying_information[2],
            "UN_sanction" :identifying_information[3]#[info.strip() for info in identifying_information if '<span' not in info]
        })

for result in results:
    print(result)


{'CELEX': ['3A32011D0236'], 'law_number': 'COUNCIL IMPLEMENTING DECISION 2011/236/CFSP', 'publication_date': ['14.4.2011'], 'legal_base': 'implementing Decision 2011/137/CFSP concerning restrictive measures in view of the situation in Libya', 'name': 'AL-BAGHDADI, Dr Abdulqader Mohammed', 'identifying_information': 'Passport number: B010574. Date of birth: 1.7.1950.', 'reason': 'Head of the Liaison Office of the Revolutionary Committees. Revolutionary Committees involved in violence against demonstrators.', 'UN_sanction': 'Date of UN designation: 26.2.2011.'}
{'CELEX': ['3A32011D0236'], 'law_number': 'COUNCIL IMPLEMENTING DECISION 2011/236/CFSP', 'publication_date': ['14.4.2011'], 'legal_base': 'implementing Decision 2011/137/CFSP concerning restrictive measures in view of the situation in Libya', 'name': 'DIBRI, Abdulqader Yusef', 'identifying_information': 'Date of birth: 1946. Place of birth: Houn, Libya.', 'reason': "Head of Muammar QADHAFI's personal security. Responsibility for r